In [1]:
from dask.distributed import Client, progress


In [2]:
client = Client(n_workers=2, threads_per_worker=2, memory_limit='2GB')
client

Client Scheduler: tcp://127.0.0.1:42049 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 4 Memory: 4.00 GB


In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import sklearn
%matplotlib inline
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from nltk.corpus import stopwords
import nltk

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/xavient/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
df = pd.read_json('News_Category_Dataset_v2.json', lines=True)
df = df.drop(['headline', 'authors', 'link', 'date'], axis=1)
df['Category'] = df['category']
df['Text'] = df['short_description']

df = df.drop(['category', 'short_description'], axis=1)
lemmatizer = nltk.stem.WordNetLemmatizer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()

def lemmatize_text(text):
    return ' '.join([str(lemmatizer.lemmatize(w)).lower() for w in w_tokenizer.tokenize(text)])

stop = stopwords.words('english')

df['Text'] = df['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df['Text'] = df['Text'].apply(lambda x: lemmatize_text(x))

df['category_id'] = df['Category'].factorize()[0]

category_id_df = df[['Category', 'category_id']].drop_duplicates().sort_values('category_id')
higher = df.groupby('Category').count()['Text'].max(axis=0)

from sklearn.utils import resample

for category in category_id_df['Category']:
    
    min_imblanced = df[df.Category==category]
    other_imblanced = df[df.Category!=category]
    
    up_scale = resample(min_imblanced, replace=True, n_samples=higher, random_state=123)
    df = pd.concat([other_imblanced, up_scale])
    #print(other_imblanced)
    
#df.head()
print(df.groupby('Category').count())

                 Text  category_id
Category                          
ARTS            32739        32739
ARTS & CULTURE  32739        32739
BLACK VOICES    32739        32739
BUSINESS        32739        32739
COLLEGE         32739        32739
COMEDY          32739        32739
CRIME           32739        32739
CULTURE & ARTS  32739        32739
DIVORCE         32739        32739
EDUCATION       32739        32739
ENTERTAINMENT   32739        32739
ENVIRONMENT     32739        32739
FIFTY           32739        32739
FOOD & DRINK    32739        32739
GOOD NEWS       32739        32739
GREEN           32739        32739
HEALTHY LIVING  32739        32739
HOME & LIVING   32739        32739
IMPACT          32739        32739
LATINO VOICES   32739        32739
MEDIA           32739        32739
MONEY           32739        32739
PARENTING       32739        32739
PARENTS         32739        32739
POLITICS        32739        32739
QUEER VOICES    32739        32739
RELIGION        3273

In [5]:
df.head()

,Category,Text,category_id
68255,CRIME,the city turn record bizarre incident.,0
59914,CRIME,"this past week, milwaukee toddler fatally shot...",0
81155,CRIME,"""a second later followed sound like car crashi...",0
151014,CRIME,to black contemporary america wrestle history ...,0
91510,CRIME,,0


In [66]:
df.Text.shape

(1342299,)

In [86]:
df = df.drop_duplicates(subset=['Text'])


In [88]:
df.Text

68255                the city turn record bizarre incident.
59914     this past week, milwaukee toddler fatally shot...
81155     "a second later followed sound like car crashi...
151014    to black contemporary america wrestle history ...
91510                                                      
                                ...                        
139827    there think poorly talkbacks. they think theat...
171645    it's easy feat getting chosen participate coun...
189735    if theater company's artistic director looking...
138547    curator rebecca rabinow awestruck discovered m...
166449    now see rotating display object sold tokyo pop...
Name: Text, Length: 162418, dtype: object

In [89]:
df.head()

,Category,Text,category_id
68255,CRIME,the city turn record bizarre incident.,0
59914,CRIME,"this past week, milwaukee toddler fatally shot...",0
81155,CRIME,"""a second later followed sound like car crashi...",0
151014,CRIME,to black contemporary america wrestle history ...,0
91510,CRIME,,0


In [90]:
df.shape

(162418, 3)

In [91]:
import dask.dataframe as dd


ddf = dd.from_pandas(pd.DataFrame({"text": df.Text, "target": df.category_id}),  npartitions=25)

In [92]:
ddf

,text,target
npartitions=25,,
0,object,int64
7747,...,...
...,...,...
193928,...,...
200852,...,...


In [93]:
ddf.head()

,text,target
0,she left husband. he killed children. just ano...,0
1,of course song.,1
2,the actor longtime girlfriend anna eberstein t...,1
3,the actor give dems ass-kicking fighting hard ...,1
4,"the ""dietland"" actress said using bag ""really ...",1


In [97]:
print(ddf.head().loc[2, 'text'])


the actor longtime girlfriend anna eberstein tied knot civil ceremony.


In [101]:
import dask_ml.feature_extraction.text

vect = dask_ml.feature_extraction.text.HashingVectorizer()
X = vect.fit_transform(ddf['text'])
X

dask.array<_transformer, shape=(nan, 1048576), dtype=float64, chunksize=(nan, 1048576), chunktype=scipy.csr_matrix>

In [102]:
X.blocks[0].compute()


<6497x1048576 sparse matrix of type '<class 'numpy.float64'>'
	with 57982 stored elements in Compressed Sparse Row format>

In [114]:
np

<module 'numpy' from '/home/xavient/Desktop/manoj/python/37_globalenv/lib/python3.7/site-packages/numpy/__init__.py'>

In [150]:
ddf['target'].isin(np.array([1])).astype(int)

Dask Series Structure:
npartitions=25
0         int64
7747        ...
          ...  
193928      ...
200852      ...
Name: target, dtype: int64
Dask Name: astype, 101 tasks

In [120]:
import numpy as np



In [121]:
import numpy as np
import sklearn.linear_model
import sklearn.pipeline

import dask_ml.wrappers

In [122]:
sgd = sklearn.linear_model.SGDClassifier(
    tol=1e-3
)
clf = dask_ml.wrappers.Incremental(
    sgd, scoring='accuracy', assume_equal_chunks=True
)
pipe = sklearn.pipeline.make_pipeline(vect, clf)

In [142]:
y = ddf['target']

In [146]:
len(y)

162418

In [155]:
type(ddf['text'])

dask.dataframe.core.Series

In [140]:
print(len(df['Text']))
print(len(df['category_id']))

162418
162418


In [157]:
pipe.fit(ddf['text'], y, incremental__classes=np.array(range(0, 41)).tolist());

In [158]:
predictions = pipe.predict(ddf['text'])
predictions

dask.array<_predict, shape=(nan,), dtype=int64, chunksize=(nan,), chunktype=numpy.ndarray>

In [161]:
predictions

dask.array<_predict, shape=(nan,), dtype=int64, chunksize=(nan,), chunktype=numpy.ndarray>

In [163]:
#dask_ml.metrics.accuracy_score(y, predictions)
